In [1]:
import numpy as np
import pandas as pd

# set some print options
np.set_printoptions(precision=4)
np.set_printoptions(threshold=5)
np.set_printoptions(suppress=True)
pd.set_option('precision', 3, 'notebook_repr_html', True, )

# init random gen
np.random.seed(2)

In [2]:
lastfm = pd.read_csv("./lastfm-dataset-360k-small/full.csv",encoding = "ISO-8859-1",index_col=0)

In [3]:
lastfm.head()

,userId,artistId,artistName,count,gender,age,country,signupDate
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137,f,22.0,Germany,"Feb 1, 2007"
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099,f,22.0,Germany,"Feb 1, 2007"
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897,f,22.0,Germany,"Feb 1, 2007"
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717,f,22.0,Germany,"Feb 1, 2007"
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706,f,22.0,Germany,"Feb 1, 2007"


## Training and Testing
Split data into training and testing datasets

In [5]:
# Subset
df = lastfm.ix[np.random.choice(lastfm.index,size=10000,replace=False)]
print (lastfm.shape)
print (df.shape)

(876783, 8)
(10000, 8)


In [6]:
# Make sure that there are at least two artists per user

twousers = pd.value_counts(df.userId, sort=False) > 1
twousers2 = twousers[twousers].index

df1 = df.select(lambda x: df.loc[x,'userId'] in twousers2)

In [7]:
def assign_test_set(df):
    sampled_ids = np.random.choice(df.index,size = np.int64(np.ceil(df.index.size*0.2)), replace = False)
    
    df.ix[sampled_ids, 'for_testing'] = True
    
    return df

In [8]:
df1['for_testing'] = False
grouped = df1.groupby('userId',group_keys=False).apply(assign_test_set)

In [9]:
lastfm_train = df1[grouped['for_testing']==False]
lastfm_test = df1[grouped['for_testing']==True]
print (lastfm_train.shape)
print (lastfm_test.shape)
print (df1.shape)

(2311, 9)
(1910, 9)
(4221, 9)


In [10]:
df1.head()

,userId,artistId,artistName,count,gender,age,country,signupDate,for_testing
782100,0b735b8d5ef3a2f3d15432587b6b616593b791a2,654188a5-bbfc-41d8-aa27-86a946dfd757,jacek kaczmarski,1137,m,30.0,Poland,"Feb 13, 2007",False
156243,0240c2c817e1ffc630fd3dda3229008d6cf4e87e,72d7d717-0837-4f2a-9641-d0f9fdd3acf7,donovan,198,m,NaN,Australia,"Sep 3, 2008",False
835613,0c4269eab84ac9282cf0a7adb9d10af4b28b9e16,84eac621-1c5a-49a1-9500-555099c6e184,spoon,92,f,21.0,United States,"Nov 24, 2006",False
809846,0bdb1ea4e384ae1f051e6867345a6685cf41754a,c1e5344e-1bff-4727-9417-a4f55e41b5ff,loreena mckennitt,16,f,56.0,Germany,"Feb 4, 2008",False
68317,00f6e6bc791caa7d8e5733259438aa8e298201c1,487bfd74-71bf-46dd-b89c-80b7a0f06f2f,the hives,65,m,NaN,United Kingdom,"Apr 7, 2006",False
